In [13]:
from selenium import webdriver
from selenium.webdriver.common.by import By
#from selenium.webdriver import Keys
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from datetime import datetime
import pytz
from bs4 import BeautifulSoup
import pandas as pd

In [14]:
url = 'https://www.apartments.com/atlanta-ga'
start = datetime.now(pytz.timezone('America/New_York')).strftime("%Y-%m-%d %I:%M %p %Z")

In [18]:
driver = webdriver.Chrome()
driver.implicitly_wait(10) # wait for 10 seconds
driver.get(url)
try:
    wait = WebDriverWait(driver, 10)
    element = wait.until(EC.presence_of_element_located((By.ID, "paging")))
    pagination = driver.find_element(By.XPATH, "/html/body/div[1]/main/section/div[1]/section[2]/div[2]/ul/li[41]/p/span").text.split()[-1]
    
finally:
    driver.quit()
pagination

'18'

In [80]:
def get_pages(pagination, url):
    for x in range(1, int(pagination)):
        #link = '/' + str(x)
        #url = url + link
        driver = webdriver.Chrome()
        #driver.implicitly_wait(10) # wait for 10 seconds
        driver.get(url + '/' + str(x))
        try:
            wait = WebDriverWait(driver, 20)
            element = wait.until(EC.presence_of_element_located((By.ID, "placardContainer")))
            yield driver.page_source
            #url.replace(link, '')
        finally:
            driver.quit()
        
        #
        
        #driver.close()

In [81]:
# loop through the pages
pages = []

for page in get_pages(pagination, url):
    pages.append(page)

len(pages)

17

In [23]:
end = datetime.now(pytz.timezone('America/New_York')).strftime("%Y-%m-%d %I:%M %p %Z")

In [333]:
soup = BeautifulSoup(pages[14], 'html')
placardContainer = soup.find('div', {'id': 'placardContainer'})
mortarwrapper = placardContainer.find_all('li', {'class':'mortar-wrapper'})
#mortarwrapper[0].find('button', {'class':'virtualTour'})['title']
#mortarwrapper[0].find('button', {'class':'videoPlay'})['data-video']
#mortarwrapper[0].find('div', {'class':"placardCarouselImgCount"}).find_all('span')[-1].text
if mortarwrapper[1].find_all('a', {'class':"property-link"})[-2].span:
    mortarwrapper[1].find_all('a', {'class':"property-link"})[-2].span.text
else:
    ...
    #print(None)
mortarwrapper[35].find_all('p')

[<p class="property-title" title="978 Myrtle St NE, Atlanta, GA"><span class="js-placardTitle title">978 Myrtle St NE</span></p>,
 <p class="property-address js-url" title="Atlanta, GA 30309">978 Myrtle St NE</p>,
 <p class="property-address js-url" title="Atlanta, GA 30309">Atlanta, GA 30309</p>,
 <p class="bed-price-range">
 <span class="property-rents">$1,500</span>
 <span class="property-beds">1 Bed</span>
 </p>]

In [338]:
master_list = []
for x in range(len(pages)):
    soup = BeautifulSoup(pages[x], 'html')
    placardContainer = soup.find('div', {'id': 'placardContainer'})
    mortarWrapper = placardContainer.find_all('li', {'class':'mortar-wrapper'})
    for item in get_listing(mortarWrapper):
        master_list.append(item)
    #break

AttributeError: 'NoneType' object has no attribute 'text'

In [337]:
def get_listing(page):
    for data in page:
        row = {}
        row['SCRAPE_TIME'] = start
        row['REQUEST_TIME'] = end
        if data.find('div', {'class':"property-title"}):
            row['LISTING_NAME'] = data.find('div', {'class':"property-title"}).text 
        else:
            row['LISTING_NAME'] = data.find('a', {'class':'property-link'})['aria-label'].split(',')[0]
                   
        row['LISTING_LINK'] = data.find('a', {'class':"property-link"})['href']
        
        meta = data.find('p', {'class':"property-beds"}).text.split(' - ')
        row['LISTING_USE'] = meta[0] if len(meta) > 1 else None 
        
        row['LISTING_ADDRESS'] = data.find('div', {'class':"property-address"}).text
        
        if data.find('p', {'class':"property-amenities"}):
            row['LISTING_DESCRIPTION'] = data.find('p', {'class':"property-amenities"}).text.strip().replace('\n',', ')
        else:
            row['LISTING_DESCRIPTION'] = None
        if data.find('button', {'class':'videoPlay'}):
            row['HAS_VIDEOS'] = True  
        else:
            row['HAS_VIDEOS'] = False
        if data.find('button', {'class':'virtualTour'}):
            row['HAS_VIRTUAL_TOUR'] = True  
        else:
            row['HAS_VIRTUAL_TOUR'] = False
        
        if data.find('button', {'class':"checkAvailability"}):
            row['HAS_EMAIL'] = True 
        else:
            row['HAS_EMAIL'] = False
        
        
        row['PRICE'] = data.find('p', {'class':"property-pricing"}).text
        if data.find('a', {'class':"phone-link"}):
            row['PHONE_NUMBER'] = data.find('a', {'class':"phone-link"}).span.text
        else:
            row['PHONE_NUMBER'] = None
       
        
        row['HAS_IMAGE'] = True if data.find('div', {'class':"placardCarouselImgCount"}) else False
        
        if data.find_all('a', {'class':"property-link"})[-2].span:
            row['SPECIAL_TEXT'] = data.find_all('a', {'class':"property-link"})[-2].span.text
        else:
            row['SPECIAL_TEXT'] = None
        
        yield row  

In [154]:
header = ['SCRAPE_TIME',
    'REQUEST_TIME',
    'LISTING_NAME',
    'LISTING_LINK',
    'LISTING_USE',
    'LISTING_ADDRESS',
    'LISTING_DESCRIPTION',
    'HAS_VIDEOS',
    'HAS_VIRTUAL_TOUR',
    'HAS_EMAIL',
    'PRICE',
    'PHONE_NUMBER',
    'HAS_IMAGE',
    'SPECIAL_TEXT']

In [277]:
def get_listing1(page):
    dataList= []
    for data in page:
        row = {}
        row['SCRAPE_TIME'] = start
        row['REQUEST_TIME'] = end
        row['LISTING_NAME'] = data.find('div', {'class':"property-title"}).text
        row['LISTING_LINK'] = data.find('a', {'class':"property-link"})['href']
        meta = data.find('p', {'class':"property-beds"}).text.split(' - ')
        row['LISTING_USE'] = meta[0] if len(meta) > 1 else None
        row['LISTING_ADDRESS'] = data.find('div', {'class':"property-address"}).text
        row['LISTING_DESCRIPTION'] = data.find('p', {'class':"property-amenities"}).text.strip().replace('\n',', ')
        video_virtual = data.find('ul', {'class':"mediaLinksList"}).text.strip().split('\n')
        row['HAS_VIDEOS'] = True if 'Videos' in video_virtual else False
        row['HAS_VIRTUAL_TOUR'] = True if 'Virtual Tour' in video_virtual else False
        check_email = data.find('button', {'class':"checkAvailability"})
        row['HAS_EMAIL'] = True if check_email.text == 'Email' else False
        row['PRICE'] = data.find('p', {'class':"property-pricing"}).text
        row['PHONE_NUMBER'] = data.find('div', {'class':"property-actions"}).span.text
        row['HAS_IMAGE'] = True if len(data.find('div', {'class':"item active us"}).img['src']) > 0 else False
        try:
            row['SPECIAL_TEXT'] = data.find_all('a', {'class':"property-link"})[-2].span.text
        except:
            row['SPECIAL_TEXT'] = None
        dataList.append(row)
    
    return dataList
    

In [339]:
len(master_list)

579

In [301]:
480 +40+40

560

In [340]:
df = pd.DataFrame(master_list, columns=header)

In [341]:
df.to_csv('data/apartments.csv', index=False)

In [342]:
df

,SCRAPE_TIME,REQUEST_TIME,LISTING_NAME,LISTING_LINK,LISTING_USE,LISTING_ADDRESS,LISTING_DESCRIPTION,HAS_VIDEOS,HAS_VIRTUAL_TOUR,HAS_EMAIL,PRICE,PHONE_NUMBER,HAS_IMAGE,SPECIAL_TEXT
0,2024-01-31 11:17 AM EST,2024-01-31 11:31 AM EST,3005 Buckhead,https://www.apartments.com/3005-buckhead-atlan...,Studio,"3005 Peachtree Rd NE, Atlanta, GA 30305","Dog & Cat Friendly, Fitness Center, Pool, Kitc...",True,True,True,"$1,737 - 3,557",(470) 870-7791,True,1 Month Free
1,2024-01-31 11:17 AM EST,2024-01-31 11:31 AM EST,Dunwoody Ridge,https://www.apartments.com/dunwoody-ridge-atla...,None,"7227 NE Peachtree Dunwoody Rd, Atlanta, GA 30328","Dog & Cat Friendly, Fitness Center, Dishwasher...",True,True,True,"$1,094 - 1,728",(844) 757-0487,True,None
2,2024-01-31 11:17 AM EST,2024-01-31 11:31 AM EST,Cortland at Phipps Plaza,https://www.apartments.com/cortland-at-phipps-...,Studio,"707 Park Ave NE, Atlanta, GA 30326","Dog & Cat Friendly, In Unit Washer & Dryer, Gr...",False,True,True,"$1,322 - 2,457",(470) 944-5573,True,None
3,2024-01-31 11:17 AM EST,2024-01-31 11:31 AM EST,Ford Factory Lofts,https://www.apartments.com/ford-factory-lofts-...,None,"699 Ponce De Leon Ave NE, Atlanta, GA 30308","Cat Friendly, Heat, Ceiling Fans, Controlled A...",True,True,True,"$1,485 - 2,800",(470) 944-5875,True,Specials
4,2024-01-31 11:17 AM EST,2024-01-31 11:31 AM EST,Calibre Woods,https://www.apartments.com/calibre-woods-atlan...,None,"2075 Lavista Rd NE, Atlanta, GA 30329","Dog & Cat Friendly, Fitness Center, Pool, Dish...",True,True,True,"$1,533 - 2,109",(470) 944-6255,True,Discounts
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
574,2024-01-31 11:17 AM EST,2024-01-31 11:31 AM EST,1939 Gotham Way NE,https://www.apartments.com/1939-gotham-way-ne-...,None,"Atlanta, GA 30324",None,False,False,False,"$4,250 /mo",(678) 904-9868,False,None
575,2024-01-31 11:17 AM EST,2024-01-31 11:31 AM EST,556 Moreland Ave NE,https://www.apartments.com/556-moreland-ave-ne...,None,"Atlanta, GA 30307",None,False,False,True,"$3,950",(470) 435-7955,False,None
576,2024-01-31 11:17 AM EST,2024-01-31 11:31 AM EST,1843 Campbellton Rd Sw,https://www.apartments.com/1843-campbellton-rd...,None,"1843 Campbellton Rd SW, Atlanta, GA 30311",None,False,False,True,"$2,260 /mo",(404) 751-2505,False,None
577,2024-01-31 11:17 AM EST,2024-01-31 11:31 AM EST,505 Whitehall St SW,https://www.apartments.com/505-whitehall-st-sw...,None,"Atlanta, GA 30303",None,False,False,False,"$1,950 - 2,600",(470) 264-3558,False,None
